In [1]:

%load_ext bigdata

In [2]:
%hive_start

In [3]:

%timeout 300

In [4]:
!hdfs dfs -ls /tmp

Found 2 items
drwxrwx---   - root supergroup          0 2020-01-20 22:44 /tmp/hadoop-yarn
drwx-wx-wx   - root supergroup          0 2020-01-20 23:11 /tmp/hive


In [5]:
!hdfs dfs -mkdir /tmp/hivep6

In [6]:
!hdfs dfs -ls /tmp/

Found 3 items
drwxrwx---   - root supergroup          0 2020-01-20 22:44 /tmp/hadoop-yarn
drwx-wx-wx   - root supergroup          0 2020-01-20 23:11 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-01-20 23:11 /tmp/hivep6


In [7]:
!hdfs dfs -copyFromLocal tbl0.csv /tmp/hivep6/

In [8]:
!hdfs dfs -copyFromLocal tbl1.csv /tmp/hivep6/

In [9]:
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/hivep6

Found 2 items
-rw-r--r--   1 root supergroup        397 2020-01-20 23:11 /tmp/hivep6/tbl0.csv
-rw-r--r--   1 root supergroup        282 2020-01-20 23:11 /tmp/hivep6/tbl1.csv


In [10]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 8.921 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 5.794 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.718 seconds


In [12]:
%%hive
SHOW TBLPROPERTIES tbl0;

SHOW TBLPROPERTIES tbl0;
OK
numFiles	1
numRows	0
rawDataSize	0
totalSize	397
transient_lastDdlTime	1579561953
Time taken: 0.103 seconds, Fetched: 5 row(s)


In [13]:
%%hive
SELECT * FROM tbl0;

SELECT * FROM tbl0;
OK
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}
3	D	6	2014-12-26	["b","d"]	{"aa":10,"bb":40}
4	D	5	2016-06-25	["a","c","e","b","d"]	{"bb":40,"dd":20,"aa":10,"cc":30}
5	C	7	2016-05-23	["d","e","c"]	{"cc":10,"aa":20}
6	A	2	2018-06-14	["a","d"]	{"aa":20}
7	B	3	2014-12-22	["a","e","d"]	{"cc":20}
8	C	6	2015-08-20	["d","a","c","e"]	{"cc":10,"aa":20}
9	B	3	2017-12-08	["b","a","c","e"]	{"cc":40,"dd":10,"aa":30,"bb":20}
10	B	7	2015-07-28	["c","d","e","a","b"]	{"aa":10,"dd":40,"cc":30}
Time taken: 1.496 seconds, Fetched: 10 row(s)


In [56]:
%%hive

DROP TABLE IF EXISTS may;

CREATE TABLE may
AS
SELECT l_may
FROM (
      SELECT c1,
             collect_list(UPPER(letra)) AS l_may
      FROM   tbl0
      LATERAL VIEW explode(c5) tbl0 AS letra
      GROUP BY c1
      ) t0

;
    
INSERT OVERWRITE DIRECTORY '/tmp/hivep6'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
SELECT * FROM may;

DROP TABLE IF EXISTS mayus;
OK
Time taken: 0.145 seconds
CREATE TABLE may
AS
SELECT l_may
FROM (
      SELECT c1,
             collect_list(UPPER(letra)) AS l_may
      FROM   tbl0
      LATERAL VIEW explode(c5) tbl0 AS letra
      GROUP BY c1
      ) t0
;
Query ID = root_20200121010841_21394cbb-94bf-467d-9bc2-630f8655e967
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1579560102290_0015, Tracking URL = http://bc1a06738ea7:8088/proxy/application_1579560102290_0015/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1579560102290_0015
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
20

In [57]:
!hdfs dfs -cat /tmp/hivep6/000000_0 

A:E:C:D
A:C
B:D
A:C:E:B:D
D:E:C
A:D
A:E:D
D:A:C:E
B:A:C:E
C:D:E:A:B


In [55]:
!hdfs dfs -copyToLocal /tmp/hivep4/000000_0 output;

In [59]:
%%hive
DROP TABLE may;
%hive_quit

DROP TABLE may;
NoViableAltException(338@[])
	at org.apache.hadoop.hive.ql.parse.HiveParser.statement(HiveParser.java:1300)
	at org.apache.hadoop.hive.ql.parse.ParseDriver.parse(ParseDriver.java:208)
	at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:77)
	at org.apache.hadoop.hive.ql.parse.ParseUtils.parse(ParseUtils.java:70)
	at org.apache.hadoop.hive.ql.Driver.compile(Driver.java:468)
	at org.apache.hadoop.hive.ql.Driver.compileInternal(Driver.java:1317)
	at org.apache.hadoop.hive.ql.Driver.runInternal(Driver.java:1457)
	at org.apache.hadoop.hive.ql.Driver.run(Driver.java:1237)
	at org.apache.hadoop.hive.ql.Driver.run(Driver.java:1227)
	at org.apache.hadoop.hive.cli.CliDriver.processLocalCmd(CliDriver.java:233)
	at org.apache.hadoop.hive.cli.CliDriver.processCmd(CliDriver.java:184)
	at org.apache.hadoop.hive.cli.CliDriver.processLine(CliDriver.java:403)
	at org.apache.hadoop.hive.cli.CliDriver.executeDriver(CliDriver.java:821)
	at org.apache.hadoop.hive.cli.CliDrive